# Weyl 群扩张群的结构

## 主要函数

1. Thetas(s,n) 返回生成元集合 \{\theta_i\}_{i=1}^n
2. OrderMatrixOfGens(gens) 返回生成元的群阶矩阵
3. GroupByOrderMathix(order_mat) 群阶矩阵作为泛性质，生成群
4. GroupTreeOfMaxDepth(G,depth) 生成群树（BFS）
5. UniversalPropertyOfGens 返回生成元的泛性质关系 

### Thetas(s,n)

In [89]:
def ExpOfNilpotentMat(mat, max_depth=36, base_ring=ZZ):
    """
    1. 返回幂零矩阵的指数矩阵
    2. max_depth 指定 x^n=0 中，n 的最大值
    3. 默认基环为整数环
    4. 输入必须为 Sage 矩阵类型
    """
    dim = mat.dimensions()[0]
    exp_mat = matrix.identity(dim) # 单位矩阵
    new,i = mat,1
    while not new.is_zero(): # 运算直到 new 为零矩阵
        exp_mat += 1/factorial(i) * new
        i += 1
        new *= mat
    return matrix(ZZ,exp_mat)

In [56]:
# 不能用自然表示，一定是伴随表示
def Thetas(s,n):
    """返回给定型的生成元集合 {\theta_i}_{i=1}^n"""
    exp = ExpOfNilpotentMat # 简写函数名
    L = LieAlgebra(ZZ,cartan_type=[s,n])
    # 一组基元
    basis = L.basis()
    # 上三角 e_i
    es = [L.e(i+1).adjoint_matrix() for i in range(n)]
    # 下三角 f_i
    fs = [L.f(i+1).adjoint_matrix() for i in range(n)]
    thetas = [exp(es[i])*exp(-fs[i])*exp(es[i]) for i in range(n)]
    return thetas

In [90]:
# gap_str = lambda s:gap('"'+s+'"')
# gap.Read(gap_str("big_weyl_group.py"))

# 函数测试
for i in range(3):
    print(len(MatrixGroup(Thetas("A",i+1))))

2
24
96


### OrderMatrixOfGens

In [96]:
def OrderMatrixOfGens(gens):
    """
    1. 计算 gens 的群阶矩阵
    2. 生成元的群基环必须为整数环
    """
    orders = [[(a*b).multiplicative_order() for a in gens] for b in gens]
    return matrix(ZZ,orders)

In [97]:
# 函数测试
gens = Thetas("A",3)
print(gens[0].base_ring())
OrderMatrixOfGens(gens)

Integer Ring


[2 3 2]
[3 2 3]
[2 3 2]

### 泛性质构造商群

In [101]:
def GroupByOrderMatrix(order_mat):
    """群阶矩阵 -> 自由群商群"""
    n = order_mat.dimensions()[0]
    FG = FreeGroup(n) # 自由群
    gens = FG.gens() # 生成元
    # 生成关系
    relations = [(gens[i]*gens[j])^order_mat[i,j] for i in range(n) for j in range(n)]
    return FG/relations

In [131]:
n = 2
gens = Thetas("A",n)
order_mat = OrderMatrixOfGens(gens)
print(order_mat)
G = GroupByOrderMatrix(order_mat)
G.1 * G.0 == G.1 * G.1

[2 3]
[3 2]


False

### 生成群树

In [183]:
def GroupTreeOfMaxDepth(G,depth=-1):
    """
    1. BFS 获取群元素，按长度分层
    2. 获取前 depth 层，首层记0，默认获取整树（有限群）"""
    # 群树，初始为零层
    tree = [[G.one()]] 
    if depth == 0: # 只有零层
        return tree
    gens = G.gens() # 生成元
    S = [G.one()] # 已生成的群元素
    tree.append([]) # 添加一层
    # 将 gens 化简（去重复元素，以及单位元）
    for a in gens:
        if a not in S:
            S.append(a)
            tree[-1].append(a)
    if depth == 1: # 只有一层
        return tree
    gens = tree[-1] # 简化的生成元
    
    # 循环开始，直到深度足够，或者群生成完毕
    while depth-1 and len(tree[-1]):
        depth -= 1
        tree.append([])
        for a in tree[-2]:
            for b in gens:
                c = a * b;
                if c not in S:
                    S.append(c)
                    tree[-1].append(c)
    return tree

In [185]:
# 测试一
mat = matrix(CoxeterMatrix(["A",2]))
print(mat)
G = GroupByOrderMatrix(mat)
GroupTreeOfMaxDepth(G)

[1 3]
[3 1]


[[1], [x0, x1], [x0*x1, x1*x0], [x0*x1*x0], []]

In [163]:
n = 2
gens = Thetas("A",n)
order_mat = OrderMatrixOfGens(gens)
print(order_mat)
G2 = GroupByOrderMatrix(order_mat)
GroupTreeOfMaxDepth(G2,3)

[2 3]
[3 2]


[[1],
 [x0, x1],
 [x0^2, x0*x1, x1*x0, x1^2],
 [x0^3, x0^2*x1, x0*x1*x0, x0*x1^2, x1*x0^2, x1*x0*x1, x1^2*x0, x1^3]]

### UniversalPropertyOfGens

In [187]:
txt = "1-1-2"
'-'.join(txt.split("-"))

'1-1-2'

In [191]:
G = MatrixGroup(Thetas("A",5))

In [196]:
[1,3,3].index(1)

0

In [165]:
def UniversalPropertyOfGroup(G,max_depth=10):
    """
    1. 求群生成元的泛性质
    2. G 的生成元不能存在相等，或者等于1
    """
    # 初始化
    gens = G.gens() # 生成元
    S = [G.one()] + gens # 已生成群元素
    assert len(S)==len(set(S)),"G 的生成元不能存在相等，或者等于1"
    
    tree = [[G.one()],gens] # 群树，初始为0层
    relations = {} # 生成关系
    
    while max_depth-1:
        max_depth -= 1
        for a in tree[-2]:
            for b in gens:
                c = a * b;
                if c not in S:
                    S.append(c)
                    tree[-1].append(c)    
    return relations

In [ ]:
def GroupTreeOfMaxDepth(G,depth=-1):
    """
    1. BFS 获取群元素，按长度分层
    2. 获取前 depth 层，首层记0，默认获取整树（有限群）"""
    # 群树，初始为零层
    tree = [[G.one()]] 
    if depth == 0: # 只有零层
        return tree
    gens = G.gens() # 生成元
    S = [G.one()] # 收集群元素
    tree.append([]) # 添加一层
    # 将 gens 化简（去重复元素，以及单位元）
    for a in gens:
        if a not in S:
            S.append(a)
            tree[-1].append(a)
    if depth == 1: # 只有一层
        return tree
    gens = tree[-1] # 简化的生成元
    
    # 循环开始，直到深度足够，或者群生成完毕
    while depth-1 and len(tree[-1]):
        depth -= 1
        tree.append([])
        for a in tree[-2]:
            for b in gens:
                c = a * b;
                if c not in S:
                    S.append(c)
                    tree[-1].append(c)
    return tree